# Prediccion de sitios web phishing

Empezaremos primero con la comprension de los datos

In [1]:
import pandas as pd


df = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv')

df.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 56 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   FILENAME                    235795 non-null  object 
 1   URL                         235795 non-null  object 
 2   URLLength                   235795 non-null  int64  
 3   Domain                      235795 non-null  object 
 4   DomainLength                235795 non-null  int64  
 5   IsDomainIP                  235795 non-null  int64  
 6   TLD                         235795 non-null  object 
 7   URLSimilarityIndex          235795 non-null  float64
 8   CharContinuationRate        235795 non-null  float64
 9   TLDLegitimateProb           235795 non-null  float64
 10  URLCharProb                 235795 non-null  float64
 11  TLDLength                   235795 non-null  int64  
 12  NoOfSubDomain               235795 non-null  int64  
 13  HasObfuscation

### Las variables mas importantes para la deteccion de phishing son:

* `IsDomaInIP`
    * Los sitios legitimos rara vez usan una direccion IP numerica en el dominio.
* `URLLength`
    * Las URL de phishing suelen ser extensas para confundir al usuario.
* `NoOfSubdomains`
    * Un numero alto de subdominios, especialmente si son sospechosos, es señal de alerta.
* `TLDLegitimateProb`
    * Algunos TLDs son más usados para phishing que otros (por ejemplo .xyz o .top).
* `HasObfuscation` y `ObfuscationRatio`
    *  La ofuscación en la URL (ej. caracteres especiales) es una táctica común de phishing para evadir la detección y confundir al usuario.
* `NoOfURLRedirect`
    * Los sitios de phishing a menudo utilizan múltiples redirecciones para dirigir a los usuarios a la página falsa, ocultando la URL original.
* `IsHTTPS` 
    * Es un factor no defenitivo, pero que puede ayudar.
* `HasExternalFormSubmit`
    *  Si un formulario de inicio de sesión envía datos a un dominio diferente al de la página actual.
* `NoOfPopup`
    * Muchos sitios de phishing utilizan popups para solicitar información o para redirigir
* `NoOfiFrame`
    * Los iframes pueden usarse para incrustar contenido malicioso o para superponer elementos sobre la página legítima
* `HasPasswordField`
    * Un sitio de phishing casi siempre tendrá un campo de contraseña para robar credenciales. La ausencia de este en un sitio que debería tenerlo (ej. banco) o su presencia en un sitio que no debería, es relevante.
* `Bank, Pay, Crypto`
    *  Estas son variables binarias que indican si el sitio contiene palabras clave muy sensibles. Los sitios de phishing se dirigen comúnmente a estas categorías.
* `SpacialCharRatioInURL`
    * Muchos caracteres especiales en la URL son sospechosos.
* `NoOfImage, NoOfCSS, NoOfJS`
    * Los sitios phishing suelen tener menos contenido o scripts.
* `label`
    * Label 1 corresponde a una URL legitima, label 0 a una URL de phishing.

In [3]:
# Filtramos el DataFrame para quedarnos con las variables que nos interesan
df = df[['Domain', 'IsDomainIP', 'URLLength', 'NoOfSubDomain', 'TLDLegitimateProb', 'HasObfuscation', 'ObfuscationRatio', 'NoOfURLRedirect',
         'IsHTTPS', 'HasExternalFormSubmit', 'NoOfPopup', 'NoOfiFrame', 'HasPasswordField', 'Bank', 'Crypto', 'Pay', 'SpacialCharRatioInURL',
         'NoOfImage', 'NoOfCSS', 'NoOfJS', 'label']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Domain                 235795 non-null  object 
 1   IsDomainIP             235795 non-null  int64  
 2   URLLength              235795 non-null  int64  
 3   NoOfSubDomain          235795 non-null  int64  
 4   TLDLegitimateProb      235795 non-null  float64
 5   HasObfuscation         235795 non-null  int64  
 6   ObfuscationRatio       235795 non-null  float64
 7   NoOfURLRedirect        235795 non-null  int64  
 8   IsHTTPS                235795 non-null  int64  
 9   HasExternalFormSubmit  235795 non-null  int64  
 10  NoOfPopup              235795 non-null  int64  
 11  NoOfiFrame             235795 non-null  int64  
 12  HasPasswordField       235795 non-null  int64  
 13  Bank                   235795 non-null  int64  
 14  Crypto                 235795 non-nu

In [4]:
df.head()

,Domain,IsDomainIP,URLLength,NoOfSubDomain,TLDLegitimateProb,HasObfuscation,ObfuscationRatio,NoOfURLRedirect,IsHTTPS,HasExternalFormSubmit,...,NoOfiFrame,HasPasswordField,Bank,Crypto,Pay,SpacialCharRatioInURL,NoOfImage,NoOfCSS,NoOfJS,label
0,www.southbankmosaics.com,0,31,1,0.522907,0,0.0,0,1,0,...,1,0,1,0,0,0.032,34,20,28,1
1,www.uni-mainz.de,0,23,1,0.032650,0,0.0,0,1,0,...,0,0,0,0,0,0.087,50,9,8,1
2,www.voicefmradio.co.uk,0,29,2,0.028555,0,0.0,0,1,0,...,0,0,0,0,0,0.069,10,2,7,1
3,www.sfnmjournal.com,0,26,1,0.522907,0,0.0,0,1,0,...,12,0,0,1,1,0.038,3,27,15,1
4,www.rewildingargentina.org,0,33,1,0.079963,0,0.0,1,1,0,...,2,0,1,0,1,0.030,244,15,34,1
